In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
from utils import serialize, deserialize
from collapse import atom_info

In [4]:
from xml_parse import parse_xml

In [5]:
import random
np.random.seed(77)
random.seed(77)

In [6]:
sites = parse_xml('../data/flat_db_file.xml')

100%|███████████████████████████████████████████████████████████████████████████████████████████████████| 27128801/27128801 [02:31<00:00, 178776.66it/s]


In [7]:
data_list = []
for s in sites:
    if s.getSiteChain().getMolType() != 'protein':
        continue
    site_id = s.getSite()
    pdb_id = s.getPDBid()
    m = s.getMetal()
    sc = s.getSiteChain()
    
    chain = m.getChainid()
    metal_pdb_name = m.getAtomName()
    # metal_pdb_atom = m.getElement()
    metal_resname = m.getResidueName()
    metal_resid = m.getResidueid()
    metal_coord_num = m.getCoord()
    site_loc = s.getSiteLoc()
    name = sc.getMolName()
    interactions = []
    
    for lig in m.getLigands():
        if lig.getEndoExo() == 'exogenous':
            exo = True
        else:
            exo = False
        resname = lig.getResidueName()
        resid = lig.getResidueid()
        m_chain = lig.getChainid()
        if resname not in atom_info.aa:
            continue
        if m_chain != chain:
            continue
        interactions.append('_'.join((resname, resid)))
    data_list.append([site_id, pdb_id, chain, name, metal_pdb_name, metal_resname, metal_resid, metal_coord_num, site_loc, interactions])

In [8]:
metal_df = pd.DataFrame(data_list, columns=['site_id', 'pdb_id', 'chain', 'protein_name', 'metal_pdb_name', 'metal_resname', 'metal_resid', 'metal_coord_num', 'site_loc', 'interactions'])

In [10]:
metal_df[(metal_df['pdb_id'] == '2c5w') & (metal_df['metal_pdb_name'] == 'ZN')]

,site_id,pdb_id,chain,protein_name,metal_pdb_name,metal_resname,metal_resid,metal_coord_num,site_loc,interactions
36988,2c5w_1,2c5w,B,Penicillin-binding protein 1A,ZN,ZN,701,2,Within a Chain,[HIS_309]
36989,2c5w_2,2c5w,B,Penicillin-binding protein 1A,ZN,ZN,703,2,Within a Chain,[HIS_460]
36990,2c5w_3,2c5w,B,Penicillin-binding protein 1A,ZN,ZN,702,3,Within a Chain,"[GLU_435, HIS_395]"
36991,2c5w_4,2c5w,B,Penicillin-binding protein 1A,ZN,ZN,705,1,Within a Chain,[HIS_349]
36992,2c5w_5,2c5w,B,Penicillin-binding protein 1A,ZN,ZN,704,1,Within a Chain,[HIS_503]


In [11]:
metal_df = metal_df[metal_df.site_loc == 'Within a Chain']

In [18]:
metal_df = metal_df.groupby(['pdb_id', 'chain', 'metal_pdb_name'])['interactions'].apply(lambda x: [subl for l in x for subl in l]).reset_index()

In [22]:
metal_df[(metal_df['pdb_id'] == '2c5w') & (metal_df['metal_pdb_name'] == 'ZN')]

,pdb_id,chain,metal_pdb_name,interactions
21160,2c5w,B,ZN,"[HIS_309, HIS_460, GLU_435, HIS_395, HIS_349, ..."


In [39]:
import collections as col
col.Counter(metal_df.interactions.apply(len))

Counter({3: 22394,
         4: 21104,
         2: 15363,
         5: 4479,
         8: 3372,
         7: 2232,
         6: 2169,
         10: 1051,
         9: 1014,
         12: 663,
         11: 410,
         16: 281,
         20: 263,
         15: 231,
         13: 203,
         14: 183,
         21: 98,
         18: 94,
         19: 77,
         17: 56,
         24: 32,
         23: 27,
         32: 24,
         22: 14,
         31: 12,
         25: 9,
         30: 9,
         29: 8,
         37: 8,
         27: 7,
         34: 6,
         26: 5,
         28: 5,
         35: 4,
         42: 3,
         36: 3,
         33: 3,
         48: 3,
         51: 2,
         40: 2,
         39: 1,
         56: 1,
         49: 1,
         46: 1,
         69: 1,
         60: 1})

In [21]:
metal_df = metal_df[metal_df.interactions.apply(len) > 1]

In [23]:
metal_df['pdb_chain'] = metal_df['pdb_id'] + metal_df['chain']

In [24]:
pdb_ec = pd.read_csv('../data/pdb_chain_enzyme.csv', skiprows=1)

In [25]:
pdb_ec['pdb_chain'] = pdb_ec['PDB'] + pdb_ec['CHAIN']

In [26]:
metal_df = pd.merge(metal_df, pdb_ec[['pdb_chain', 'EC_NUMBER']], how='left')

In [42]:
import json
with open('../../pdb/pdb_to_cath.json', 'r') as f:
    pdb_to_cath = json.load(f)

In [84]:
import collections as col
cath_to_pdb = col.defaultdict(list)
for p, cathlist in pdb_to_cath.items():
    for c in cathlist:
        cath_to_pdb[c].append(p)

In [44]:
metal_df['CATH'] = metal_df['pdb_id'].map(pdb_to_cath)

In [57]:
metal_single_domain = metal_df[metal_df['CATH'].apply(lambda x: len(x) if isinstance(x, list) else 0) == 1]
metal_single_domain['CATH'] = [x[0] for x in metal_single_domain['CATH']]

In [79]:
pd.merge(metal_single_domain[metal_single_domain.metal_pdb_name == 'ZN'].value_counts('EC_NUMBER'), metal_single_domain[metal_single_domain.metal_pdb_name != 'ZN'].value_counts('EC_NUMBER'), left_index=True, right_index=True)

,count_x,count_y
EC_NUMBER,,
4.2.1.1,957,171
2.3.2.27,352,9
1.15.1.1,307,265
3.4.24.-,189,180
1.14.11.66,160,126
...,...,...
6.5.1.1,1,2
7.1.1.9,1,1
2.4.1.33,1,1


In [92]:
zn_single_domain = metal_single_domain[(metal_single_domain['metal_pdb_name'] == 'ZN') & ~metal_single_domain['EC_NUMBER'].isna()]

In [93]:
for cath, df in zn_single_domain.groupby('CATH'):
    print(df[['pdb_chain', 'CATH', 'EC_NUMBER', 'interactions']])
    pdb_matches = cath_to_pdb[cath]
    other_pdb = list(set(pdb_matches) - set(df['pdb_id'].tolist()))
    print(other_pdb[:5])
    break

      pdb_chain          CATH EC_NUMBER      interactions
58753     4zyaA  1.10.10.1430  6.1.1.22  [GLU_39, HIS_71]
58754     4zyaB  1.10.10.1430  6.1.1.22  [HIS_71, GLU_39]
['2kqr']


In [27]:
metal_enzymes = metal_df[~metal_df['EC_NUMBER'].isna()]
metal_enzymes = metal_enzymes[metal_enzymes['metal_pdb_name'] != '']

In [28]:
metal_enzymes.metal_pdb_name.value_counts()[:8]

metal_pdb_name
ZN    15088
MG     9371
CA     8009
NA     5006
FE     3287
MN     3012
K      2081
CU     1339
Name: count, dtype: int64

In [37]:
metal_enzymes[(metal_enzymes['pdb_id'] == '2c5w') & (metal_enzymes['metal_pdb_name'] == 'ZN')]

,pdb_id,chain,metal_pdb_name,interactions,pdb_chain,EC_NUMBER,resolution
15087,2c5w,B,ZN,"[HIS_309, HIS_460, GLU_435, HIS_395, HIS_349, ...",2c5wB,2.4.1.129,2.55
15088,2c5w,B,ZN,"[HIS_309, HIS_460, GLU_435, HIS_395, HIS_349, ...",2c5wB,3.4.16.4,2.55


In [41]:
metal_enzymes.groupby('EC_NUMBER')['metal_pdb_name'].unique()

EC_NUMBER
1.-.-.-      [PT, FE, ZN, NA, NI, CA, CU]
1.1.1.-      [NA, ZN, FE, CD, MG, CA, MN]
1.1.1.1       [ZN, CA, CD, K, FE, CU, CO]
1.1.1.100            [CA, CS, MG, NI, NA]
1.1.1.103                    [ZN, CA, NA]
                         ...             
7.6.2.-                          [NA, MG]
7.6.2.1                          [HG, MG]
7.6.2.2                          [MG, HG]
7.6.2.3                              [MG]
7.6.2.8                              [MG]
Name: metal_pdb_name, Length: 1874, dtype: object

In [29]:
pdb_resol = {}
with open('../data/resolu.idx') as f:
    for line in f:
        l = [x.strip() for x in line.strip().split(';')]
        if (len(l) != 2) or (l[1] == ''):
            continue
        pdb_resol[l[0]] = float(l[1])

In [30]:
metal_enzymes['resolution'] = metal_enzymes['pdb_id'].apply(lambda x: pdb_resol.get(x.upper()))

In [31]:
ions = ['ZN', 'MG', 'CA', 'FE', 'NA', 'MN', 'K']

In [32]:
database = {k:{} for k in ions}
for el in ions:
    subdf = metal_enzymes[metal_enzymes['metal_pdb_name'] == el]
    pos_pdb = subdf.sort_values('resolution').groupby('EC_NUMBER')['pdb_chain'].nth(0)
    pos_df = subdf[subdf['pdb_chain'].isin(pos_pdb)]
    database[el]['pos'] = pos_df
    
    neg_df = metal_enzymes[(metal_enzymes['metal_pdb_name'] != el) & (~metal_enzymes['pdb_chain'].isin(pos_pdb))]
    neg_pdb = neg_df.sort_values('resolution').groupby('EC_NUMBER')['pdb_chain'].nth(0)
    neg_pdb = neg_pdb.sample(len(pos_pdb))
    assert len(set(neg_pdb) & set(pos_pdb)) == 0
    neg_df = neg_df[neg_df['pdb_chain'].isin(neg_pdb)]
    # neg_df = neg_df.sample(len(pos_df))
    database[el]['neg'] = neg_df
    
    print(el, len(pos_df), len(neg_df))

ZN 756 867
MG 776 883
CA 660 781
FE 228 292
NA 677 844
MN 295 353
K 276 372


In [33]:
serialize(database, '../data/metal_database_balanced_2.pkl')

### Create embeddings database

In [34]:
from sklearn.model_selection import train_test_split
from atom3d.datasets import load_dataset
from collapse import process_pdb, embed_protein, initialize_model, atom_info
from torch_geometric.nn import knn_graph
import torch

In [35]:
database = deserialize('../data/metal_database_balanced_2.pkl')

In [36]:
db = database['ZN']
db_pos = db['pos']
db_pos[db_pos['pdb_id']=='2c5w']

,pdb_id,chain,metal_pdb_name,interactions,pdb_chain,EC_NUMBER,resolution
15087,2c5w,B,ZN,"[HIS_309, HIS_460, GLU_435, HIS_395, HIS_349, ...",2c5wB,2.4.1.129,2.55
15088,2c5w,B,ZN,"[HIS_309, HIS_460, GLU_435, HIS_395, HIS_349, ...",2c5wB,3.4.16.4,2.55


In [38]:
db_pos

,pdb_id,chain,metal_pdb_name,interactions,pdb_chain,EC_NUMBER,resolution
41,1a2p,C,ZN,"[LYS_62, GLU_60]",1a2pC,3.1.27.-,1.50
241,1ak0,A,ZN,"[GLU_181, HIS_15]",1ak0A,3.1.30.1,1.80
373,1atl,A,ZN,"[HIS_142, HIS_146, HIS_152]",1atlA,3.4.24.42,1.80
427,1aye,A,ZN,"[GLU_72, HIS_196, HIS_69]",1ayeA,3.4.17.15,1.80
768,1bqb,A,ZN,"[HIS_148, GLU_168, HIS_144]",1bqbA,3.4.24.29,1.72
...,...,...,...,...,...,...,...
75065,6h0t,A,ZN,"[HIS_168, HIS_48, SER_194, GLU_78, ASP_77, ASP...",6h0tA,3.1.1.6,1.90
75177,6h9j,A,ZN,"[ASP_201, HIS_279, HIS_199]",6h9jA,1.14.11.30,1.83
75178,6h9j,A,ZN,"[ASP_201, HIS_279, HIS_199]",6h9jA,1.14.11.n4,1.83
75419,6j4t,A,ZN,"[ASN_295, HIS_240, HIS_15, HIS_217, HIS_13]",6j4tA,3.5.4.-,1.82


In [52]:
for p in list(db['pos']['pdb_id'].unique()) + list(db['neg']['pdb_id'].unique()):
    if not os.path.exists('../data/pdb/' + p + '.cif'):
        os.system('wget https://files.rcsb.org/download/' + p + '.cif -O ../data/pdb/' + p + '.cif')
    break

--2023-09-17 19:03:56--  https://files.rcsb.org/download/1atl.cif
Resolving files.rcsb.org (files.rcsb.org)... 132.249.210.234
Connecting to files.rcsb.org (files.rcsb.org)|132.249.210.234|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘../data/pdb/1atl.cif’

     0K .......... .......... .......... .......... .......... 2.05M
    50K .......... .......... .......... .......... .......... 4.02M
   100K .......... .......... .......... .......... ..........  128M
   150K .......... .......... .......... .......... ..........  119M
   200K .......... .......... .......... .......... .......... 4.29M
   250K .......... .......... .......... .......... ..........  121M
   300K .......... .......... .......... .......... ..........  102M
   350K .......... .                                            139M=0.05s

2023-09-17 19:03:56 (7.17 MB/s) - ‘../data/pdb/1atl.cif’ saved [369667]



In [75]:
import atom3d.util.formats as fo
from atom3d.filters.filters import first_model_filter
def process_pdb(pdb_file, chain=None, include_hets=True):
    atoms = fo.bp_to_df(fo.read_mmcif(pdb_file, name=None, auth_residues=False))
    atoms = first_model_filter(atoms)
    if chain:
        atoms = atoms[atoms.chain == chain]
    atoms = atoms[atoms.resname != 'HOH']
    atoms = atoms[atoms.element != 'H'].reset_index(drop=True)
    if not include_hets:
        atoms = atoms[atoms.resname.isin(atom_info.aa)].reset_index(drop=True)
    return atoms

In [76]:
process_pdb('../data/pdb/2kpn.cif', chain='A', include_hets=False).head()#[['resname', 'residue']].iloc[:20]

TypeError: 'type' object cannot be interpreted as an integer

In [26]:
db = database['K']
db_pos = db['pos']
n_pos = len(db_pos)
num_pos_ec = db_pos['EC_NUMBER'].unique()
train_ec, test_ec = train_test_split(num_pos_ec, test_size=0.2, random_state=77)
train_df = db_pos[db_pos['EC_NUMBER'].isin(train_ec)]
test_df = db_pos[db_pos['EC_NUMBER'].isin(test_ec)]
train_keyres = dict(train_df.groupby('pdb_chain')['interactions'].apply(lambda x: [i for i in sum(x, []) if i.split('_')[0] in aa]))
test_keyres = dict(test_df.groupby('pdb_chain')['interactions'].apply(lambda x: [i for i in sum(x, []) if i.split('_')[0] in aa]))

In [42]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = initialize_model(device=device)

def compute_adjacency(df, resids, num_neighbors=8):
    df = df[df.name == 'CA']
    df['resid'] = df['resname'].apply(lambda x: atom_info.aa_to_letter(x)) + df['residue'].astype(str) + df['insertion_code'].astype(str).str.strip()
    df = df[df.resid.isin(resids)]
    edge_index = knn_graph(torch.tensor(df[['x', 'y', 'z']].to_numpy()), k=num_neighbors) # COO format
    return edge_index.numpy()

def create_graph_from_pdb(pdb_chain, pdb_dir, model, device):
    pdb, chain = pdb_chain[:4], pdb_chain[-1]
    atom_df = process_pdb(os.path.join(pdb_dir, pdb[1:3], f'pdb{pdb}.ent.gz'), chain=chain, include_hets=False)
    outdata = embed_protein(atom_df.copy(), model, device=device, include_hets=False, env_radius=10.0)
    if outdata is None:
        return
    outdata['adj'] = compute_adjacency(atom_df.copy(), outdata['resids'])
    return elem

In [43]:
pdb_dir = '/scratch/users/aderry/pdb'
for pdbc, res in train_keyres.items():
    emb_data = create_graph_from_pdb(pdbc, pdb_dir, model, device)
    print(emb_data)
    break

NameError: name 'self' is not defined